In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden GenAI Workshop for Image Generation (inspired by Stable Diffusion WebUI)

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_stable_diffusion_gradio.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_gradio.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates starting a playground based on [Gradio UI](https://www.gradio.app/), inspired by the popular [Stable Diffusion WebUI](https://github.com/AUTOMATIC1111/stable-diffusion-webui) project, which allows users to interact with the stable diffusion models more easily and intuitively. The playground now support `text-to-image`, `image-inpainting`, `controlnet-canny`, `instruct-pix2pix`, and `SD 4x upscaler` tasks.

This notebook also supports Dreambooth finetune the stable diffusion 1.5/2.1 models.

### Objective

- Deploy model to a [Vertex AI Endpoint resource](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for `text-to-image`, `image-inpainting`, `controlnet-canny`, `instruct-pix2pix`, and `SD 4x upscaler` tasks, from the UI.
- Adjust the parameters, such as prompt, negative_prompt, num_inference_steps, and check out the generated images for best image quality.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Run the playground

In [ ]:
# @title Setup Google Cloud project and prepare the dependencies

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

BUCKET_URI = "gs://"  # @param {type:"string"}

# @markdown 3. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

# @markdown 4. If you want to run predictions with A100 80GB or H100 GPUs, we recommend using the regions listed below. **NOTE:** Make sure you have associated quota in selected regions. Click the links to see your current quota for each GPU type: [Nvidia A100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_a100_80gb_gpus), [Nvidia H100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_h100_gpus). You can request for quota following the instructions at ["Request a higher quota"](https://cloud.google.com/docs/quota/view-manage#requesting_higher_quota).

# @markdown | Machine Type | Accelerator Type | Recommended Regions |
# @markdown | ----------- | ----------- | ----------- |
# @markdown | a2-ultragpu-1g | 1 NVIDIA_A100_80GB | us-central1, us-east4, europe-west4, asia-southeast1, us-east4 |
# @markdown | a3-highgpu-2g | 2 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-4g | 4 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-8g | 8 NVIDIA_H100_80GB | us-central1, europe-west4, us-west1, asia-southeast1 |

! pip3 install --upgrade gradio==4.29.0 opencv-python
# Uninstall nest-asyncio and uvloop as a workaround to https://github.com/gradio-app/gradio/issues/8238#issuecomment-2101066984
! pip3 uninstall --yes nest-asyncio uvloop
# A workaround for the compatibility between the fastapi and pydantic
! pip3 install fastapi==0.112.3

import datetime
import glob
import importlib
import os
import shutil
import uuid
from itertools import chain

import cv2
import gradio as gr
import numpy as np
from google.cloud import aiplatform, storage
from PIL import Image, ImageOps

! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git
common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    if not os.environ.get("GOOGLE_CLOUD_REGION"):
        raise ValueError(
            "REGION must be set. See"
            " https://cloud.google.com/vertex-ai/docs/general/locations for"
            " available cloud locations."
        )
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "stable_diffusion")


# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/storage.admin"
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/aiplatform.user"

# @markdown Set use_dedicated_endpoint to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint). Note that [dedicated endpoint does not support VPC Service Controls](https://cloud.google.com/vertex-ai/docs/predictions/choose-endpoint-type), uncheck the box if you are using VPC-SC.
use_dedicated_endpoint = True  # @param {type:"boolean"}

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [ ]:
# @title Start the playground

# @markdown This is a playground for image generation similar to the popular [Stable Diffusion WebUI](https://github.com/AUTOMATIC1111/stable-diffusion-webui).
# @markdown After the cell runs, this playground is available in a separate browser tab if you click the public URL.
# @markdown Something similar to ["https://####.gradio.live"](#) in the output of the cell.

# @markdown For Image generations, five tasks `text-to-image`, `image-inpainting`, `controlnet-canny`, `instruct-pix2pix` and `SD 4x upscaler` are currently supported.

# @markdown **How to use:**
# @markdown 1. Important: Notebook cell reruns create new public URLs. Previous URLs will stop working.
# @markdown 1. Before you start, you need to select a Vertex prediction endpoint, with a matching model deployed to the endpoint
# @markdown from the endpoint dropdown list, that has been deployed in the project and region;
# @markdown 1. Make sure the selected endpoint/model match with the chosen task. Mismatched task and model will produce unreliable results.
# @markdown 1. If no models were deployed in the past, you can create a new Vertex prediction
# @markdown endpoint by selecting your favorite model and click "Deploy".
# @markdown 1. New model deployment takes ~20 minutes. You can check the progress at [Vertex Online Prediction](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).
# @markdown 1. Adjust the prompt/negative-prompt, image-dimension, inference steps, guidance-scale to achieve the optimum image quality and inference latency.
# @markdown 1. Don't forget to undeploy the models after all the experiment to avoid continuous charges to the project.

# @markdown **Note: we support the following models now:**
# @markdown Other models (with the same task) may work, but they are not tested so use with caution.
# @markdown 1. Text-to-Image models
# @markdown    > stabilityai/stable-diffusion-2-1 \
# @markdown    > stabilityai/stable-diffusion-xl-base-1.0 \
# @markdown    > stabilityai/stable-diffusion-xl-base-1.0 - refiner \
# @markdown    > latent-consistency/lcm-sdxl \
# @markdown    > latent-consistency/lcm-lora-sdxl \
# @markdown    > stabilityai/sdxl-turbo \
# @markdown    > runDiffusion/Juggernaut-XL-Lightning \
# @markdown    > stablediffusionapi/juggernaut-xl-v9 \
# @markdown    > lykon/dreamshaper-xl-v2-turbo \
# @markdown    > stablediffusionapi/anything-xl \
# @markdown    > stablediffusionapi/turbovision-xl \
# @markdown    > bytedance/sdxl-lightning
# @markdown 1. Image-inpainting models
# @markdown    > kandinsky-community/kandinsky-2-2-decoder-inpaint \
# @markdown    > diffusers/stable-diffusion-xl-1.0-inpainting-0.1
# @markdown 1. Controlnet-canny models
# @markdown    > lllyasviel/sd-controlnet-canny
# @markdown 1. Instruct-pix2pix models
# @markdown    > timbrooks/instruct-pix2pix
# @markdown 1. SD 4x upscaler models
# @markdown    > stabilityai/stable-diffusion-x4-upscaler

# @markdown For dreambooth finetune task, we now support finetuning from the following base models: \
# @markdown 1. Stable diffusion text to image models
# @markdown    > stabilityai/stable-diffusion-2-1 with a resolution of 512x512 \
# @markdown    > stabilityai/stable-diffusion-2-1 with a resolution of 768x768 \

style_list = [
    {
        "name": "(No style)",
        "prompt": "{prompt}",
        "negative_prompt": "",
    },
    {
        "name": "Cinematic",
        "prompt": "cinematic still {prompt} . emotional, harmonious, vignette, highly detailed, high budget, bokeh, cinemascope, moody, epic, gorgeous, film grain, grainy",
        "negative_prompt": "anime, cartoon, graphic, text, painting, crayon, graphite, abstract, glitch, deformed, mutated, ugly, disfigured",
    },
    {
        "name": "Photographic",
        "prompt": "cinematic photo {prompt} . 35mm photograph, film, bokeh, professional, 4k, highly detailed",
        "negative_prompt": "drawing, painting, crayon, sketch, graphite, impressionist, noisy, blurry, soft, deformed, ugly",
    },
    {
        "name": "Anime",
        "prompt": "anime artwork {prompt} . anime style, key visual, vibrant, studio anime,  highly detailed",
        "negative_prompt": "photo, deformed, black and white, realism, disfigured, low contrast",
    },
    {
        "name": "Manga",
        "prompt": "manga style {prompt} . vibrant, high-energy, detailed, iconic, Japanese comic style",
        "negative_prompt": "ugly, deformed, noisy, blurry, low contrast, realism, photorealistic, Western comic style",
    },
    {
        "name": "Digital Art",
        "prompt": "concept art {prompt} . digital artwork, illustrative, painterly, matte painting, highly detailed",
        "negative_prompt": "photo, photorealistic, realism, ugly",
    },
    {
        "name": "Pixel art",
        "prompt": "pixel-art {prompt} . low-res, blocky, pixel art style, 8-bit graphics",
        "negative_prompt": "sloppy, messy, blurry, noisy, highly detailed, ultra textured, photo, realistic",
    },
    {
        "name": "Fantasy art",
        "prompt": "ethereal fantasy concept art of  {prompt} . magnificent, celestial, ethereal, painterly, epic, majestic, magical, fantasy art, cover art, dreamy",
        "negative_prompt": "photographic, realistic, realism, 35mm film, dslr, cropped, frame, text, deformed, glitch, noise, noisy, off-center, deformed, cross-eyed, closed eyes, bad anatomy, ugly, disfigured, sloppy, duplicate, mutated, black and white",
    },
    {
        "name": "Neonpunk",
        "prompt": "neonpunk style {prompt} . cyberpunk, vaporwave, neon, vibes, vibrant, stunningly beautiful, crisp, detailed, sleek, ultramodern, magenta highlights, dark purple shadows, high contrast, cinematic, ultra detailed, intricate, professional",
        "negative_prompt": "painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured",
    },
    {
        "name": "3D Model",
        "prompt": "professional 3d model {prompt} . octane render, highly detailed, volumetric, dramatic lighting",
        "negative_prompt": "ugly, deformed, noisy, low poly, blurry, painting",
    },
]

styles = {k["name"]: (k["prompt"], k["negative_prompt"]) for k in style_list}
STYLE_NAMES = list(styles.keys())
DEFAULT_STYLE_NAME = "(No style)"


def get_bucket_and_blob_name(filepath):
    # The gcs path is of the form gs://
    gs_suffix = filepath.split("gs://", 1)[1]
    return tuple(gs_suffix.split("/", 1))


def canny(image, low_threshold=100, high_threshold=200) -> Image.Image:
    image = np.array(image)
    image = cv2.Canny(image, low_threshold, high_threshold)

    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    image = Image.fromarray(image)
    return image


def upload_local_dir_to_gcs(local_dir_path, gcs_dir_path):
    """Uploads files in a local directory to a GCS directory."""
    client = storage.Client()
    bucket_name = gcs_dir_path.split("/")[2]
    bucket = client.get_bucket(bucket_name)
    for local_file in glob.glob(local_dir_path + "/**"):
        if not os.path.isfile(local_file):
            continue
        filename = local_file[1 + len(local_dir_path) :]
        gcs_file_path = os.path.join(gcs_dir_path, filename)
        _, blob_name = get_bucket_and_blob_name(gcs_file_path)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(local_file)
        print("Copied {} to {}.".format(local_file, gcs_file_path))


def is_image_generation_endpoint(endpoint: aiplatform.Endpoint) -> bool:
    """Returns True if the endpoint is an image generation endpoint."""
    return (
        "sd" in endpoint.display_name.lower()
        or "diffusion" in endpoint.display_name.lower()
        or "inpaint" in endpoint.display_name.lower()
        or "controlnet" in endpoint.display_name.lower()
        or "pix2pix" in endpoint.display_name.lower()
        or "upscaler" in endpoint.display_name.lower()
        or "canny" in endpoint.display_name.lower()
        or "gradio" in endpoint.display_name.lower()
    )


def list_endpoints() -> list[str]:
    """Returns all valid prediction endpoints for in the project and region."""
    # Gets all the valid endpoints in the project and region.
    endpoints = aiplatform.Endpoint.list(order_by="create_time desc")
    # Filters out the endpoints which do not have a deployed model, and the endpoint is for image generation
    endpoints = list(
        filter(
            lambda endpoint: endpoint.traffic_split
            and is_image_generation_endpoint(endpoint),
            endpoints,
        )
    )

    endpoint_names = list(
        map(
            lambda endpoint: f"{endpoint.name} - {endpoint.display_name[:40]}",
            endpoints,
        )
    )

    return endpoint_names


def get_endpoint(endpoint_name: str) -> aiplatform.Endpoint:
    """Returns a Vertex endpoint for the given endpoint_name."""

    endpoint_id = endpoint_name.split(" - ")[0]
    endpoint = aiplatform.Endpoint(
        f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_id}"
    )

    return endpoint


def get_publisher_model_id(model_name: str) -> str | None:
    """Returns the corresponding task name for the given model_name."""

    model_to_publisher_dict = {
        "stabilityai/stable-diffusion-2-1": ("stability-ai", "stable-diffusion-2-1"),
        "stabilityai/stable-diffusion-xl-base-1.0": (
            "stability-ai",
            "stable-diffusion-xl-base",
        ),
        "stabilityai/stable-diffusion-xl-base-1.0 - refiner": (
            "stability-ai",
            "stable-diffusion-xl-base",
        ),
        "latent-consistency/lcm-sdxl": ("stability-ai", "stable-diffusion-xl-lcm"),
        "latent-consistency/lcm-lora-sdxl": ("stability-ai", "stable-diffusion-xl-lcm"),
        "stabilityai/sdxl-turbo": None,
        "bytedance/sdxl-lightning": ("bytedance", "stable-diffusion-xl-lightning"),
        "runDiffusion/Juggernaut-XL-Lightning": None,
        "stablediffusionapi/juggernaut-xl-v9": None,
        "lykon/dreamshaper-xl-v2-turbo": None,
        "stablediffusionapi/anything-xl": None,
        "stablediffusionapi/turbovision_xl": None,
        "kandinsky-community/kandinsky-2-2-decoder-inpaint": None,
        "diffusers/stable-diffusion-xl-1.0-inpainting-0.1": (
            "runwayml",
            "stable-diffusion-inpainting",
        ),
        "timbrooks/instruct-pix2pix": ("timbrooks", "instruct-pix2pix"),
        "lllyasviel/sd-controlnet-canny": ("lllyasviel", "control-net"),
        "stabilityai/stable-diffusion-x4-upscaler": (
            "stability-ai",
            "stable-diffusion-4x-upscaler",
        ),
    }

    if model_name not in model_to_publisher_dict.keys():
        print(model_name)
        raise gr.Error("Select a valid model name for Endpoint creation.")

    if model_to_publisher_dict[model_name]:
        publisher = model_to_publisher_dict[model_name][0]
        publisher_model_id = model_to_publisher_dict[model_name][1]
    else:
        publisher = "hf-" + model_name.split("/")[0]
        publisher_model_id = model_name.split("/")[1]
    return f"publishers/{publisher}/models/{publisher_model_id}"


def get_task_name(model_name: str) -> str:
    """Returns the corresponding task name for the given model_name."""

    model_to_task_dict = {
        "stabilityai/stable-diffusion-2-1": "text-to-image",
        "stabilityai/stable-diffusion-xl-base-1.0": "text-to-image-sdxl",
        "stabilityai/stable-diffusion-xl-base-1.0 - refiner": "text-to-image-refiner",
        "latent-consistency/lcm-sdxl": "text-to-image-sdxl-lcm",
        "latent-consistency/lcm-lora-sdxl": "text-to-image-sdxl-lcm-lora",
        "stabilityai/sdxl-turbo": "text-to-image-sdxl-turbo",
        "bytedance/sdxl-lightning": "text-to-image-sdxl-lightning",
        "runDiffusion/Juggernaut-XL-Lightning": "text-to-image-custom",
        "stablediffusionapi/juggernaut-xl-v9": "text-to-image-custom",
        "lykon/dreamshaper-xl-v2-turbo": "text-to-image-custom",
        "stablediffusionapi/anything-xl": "text-to-image-custom",
        "stablediffusionapi/turbovision_xl": "text-to-image-custom",
        "kandinsky-community/kandinsky-2-2-decoder-inpaint": "image-inpainting",
        "diffusers/stable-diffusion-xl-1.0-inpainting-0.1": "image-inpainting",
        "timbrooks/instruct-pix2pix": "instruct-pix2pix",
        "lllyasviel/sd-controlnet-canny": "controlnet",
        "stabilityai/stable-diffusion-x4-upscaler": "conditioned-super-res",
    }

    if model_name not in model_to_task_dict.keys():
        print(model_name)
        raise gr.Error("Select a valid model name for Endpoint creation.")

    return model_to_task_dict[model_name]


def deploy_model(model_name: str) -> aiplatform.Endpoint:
    """Preprocess the model_name, before deploying the model to vertex."""
    if not model_name:
        raise gr.Error("Select a valid model name for model list.")

    model_id = model_name.split(": ")[1]
    publisher_model_id = get_publisher_model_id(model_id)
    task_name = get_task_name(model_id)

    return deploy_model_vertex(
        model_id, publisher_model_id, task_name, lora_id, use_dedicated_endpoint
    )


def deploy_model_vertex(
    model_id: str,
    publisher_model_id: str | None,
    task_name: str,
    lora_id: str = "",
    use_dedicated_endpoint: bool = False,
) -> aiplatform.Endpoint:
    """
    Creates a new Vertex prediction endpoint and deploys a model to it.
    The `model_id` can either be a HF model resource id, or a GCS path where
    the model checkpoint is saved (usually after finetune).
    """
    refiner_model_id = "stabilityai/stable-diffusion-xl-refiner-1.0"

    gr.Info("Model deployment started. Let's wait...")

    display_name = common_util.create_job_name(model_id)
    deploy_model_id = model_id
    if (
        model_id == "stabilityai/stable-diffusion-xl-base-1.0 - refiner"
        or model_id == "latent-consistency/lcm-sdxl"
        or model_id == "latent-consistency/lcm-lora-sdxl"
    ):
        deploy_model_id = "stabilityai/stable-diffusion-xl-base-1.0"

    endpoint = aiplatform.Endpoint.create(
        display_name=display_name,
        dedicated_endpoint_enabled=use_dedicated_endpoint,
    )
    serving_env = {
        "MODEL_ID": deploy_model_id,
        "TASK": task_name,
        "DEPLOY_SOURCE": "notebook_gradio",
    }
    if model_id == "stabilityai/stable-diffusion-xl-base-1.0 - refiner":
        serving_env = {
            **serving_env,
            "REFINER_MODEL_ID": refiner_model_id,
        }
    if lora_id:
        serving_env = {
            **serving_env,
            "LORA_ID": lora_id,
        }

    if "text-to-image" in task_name:
        model = upload_model_pytorch_inference_container(
            model_id, publisher_model_id, serving_env
        )
    else:
        model = upload_model_pytorch_diffusers_serve_container(
            model_id, publisher_model_id, serving_env
        )

    machine_type = "g2-standard-8"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 1

    common_util.check_quota(
        project_id=PROJECT_ID,
        region=REGION,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        is_for_training=False,
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        sync=False,
        system_labels={
            "NOTEBOOK_NAME": "model_garden_pytorch_stable_diffusion_gradio.ipynb"
        },
    )

    gr.Info(
        f"Model {display_name} is being deployed. It may take ~20 minutes to complete."
    )

    return endpoint


# The pre-built vertex-model-garden/pytorch-inference docker.
PYTORCH_INFERENCE_DOCKER_URI = "us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/pytorch-inference.cu125.0-4.ubuntu2204.py310"


def upload_model_pytorch_inference_container(
    model_id: str, publisher_model_id: str | None, serving_env: dict
) -> aiplatform.Model:
    return aiplatform.Model.upload(
        display_name=model_id,
        serving_container_image_uri=PYTORCH_INFERENCE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predict",
        serving_container_health_route="/health",
        serving_container_environment_variables=serving_env,
        model_garden_source_model_name=publisher_model_id,
    )


# The pre-built serving docker image. They contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve-opt:20240605_1400_RC00"


def upload_model_pytorch_diffusers_serve_container(
    model_id: str, publisher_model_id: str | None, serving_env: dict
) -> aiplatform.Model:
    return aiplatform.Model.upload(
        display_name=model_id,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        model_garden_source_model_name=publisher_model_id,
    )


def get_default_dimension(model_name: str) -> int:
    """Returns the default dimension for the given model_name."""

    dimension = 512
    if not model_name:
        return dimension

    model_name = model_name.lower()
    if (
        "stable-diffusion-xl" in model_name
        or "xl" in model_name
        or "lightning" in model_name
    ):
        dimension = 1024
    if "stable-diffusion-2-1" in model_name:
        dimension = 768
    if "sdxl-turbo" in model_name:
        dimension = 512

    return dimension


def get_default_guidance_scale(model_name: str) -> int:
    """Returns the default guidance scale for the given model_name."""

    guidance_scale = 7.5
    if not model_name:
        return guidance_scale

    model_name = model_name.lower()
    if "xl-base" in model_name or "refiner" in model_name:
        guidance_scale = 7.5

    if "lcm" in model_name or "sdxl-turbo" in model_name:
        guidance_scale = 0

    if "sdxl-lightning" in model_name:
        guidance_scale = 1

    if "dreamshaper-xl-v2-turbo" in model_name:
        guidance_scale = 2

    if "lightning" in model_name or "turbovision_xl" in model_name:
        guidance_scale = 1.5

    return guidance_scale


def get_default_num_inference_steps(model_name: str) -> int:
    """Returns the default num_inference_steps for the given model_name."""

    num_inference_steps = 25
    if not model_name:
        return num_inference_steps

    model_name = model_name.lower()
    if "lcm" in model_name:
        num_inference_steps = 8
    elif "sdxl-turbo" in model_name:
        num_inference_steps = 2
    elif "juggernaut-xl-lightning" in model_name:
        num_inference_steps = 5
    elif "dreamshaper-xl-v2-turbo" in model_name:
        num_inference_steps = 8
    elif "juggernaut-xl-v9" in model_name:
        num_inference_steps = 25
    elif "turbovision_xl" in model_name:
        num_inference_steps = 5
    elif "sdxl-lightning" in model_name:
        num_inference_steps = 4

    return num_inference_steps


def apply_style(style_name: str, positive: str, negative: str = "") -> tuple[str, str]:
    p, n = styles.get(style_name, styles[DEFAULT_STYLE_NAME])
    return p.replace("{prompt}", positive), n + negative


def generate_images(
    endpoint_name,
    style_name=None,
    prompt="",
    negative_prompt="",
    num_samples=1,
    guidance_scale=7.5,
    num_inference_steps=25,
    image_dimension=512,
) -> list[Image.Image]:
    if not endpoint_name:
        raise gr.Error("Select (or deploy) a model first!")

    prompt, negative_prompt = apply_style(style_name, prompt, negative_prompt)

    instances = [{"text": prompt}]
    parameters = {
        "negative_prompt": negative_prompt,
        "height": image_dimension,
        "width": image_dimension,
        "guidance_scale": guidance_scale,
        "num_inference_steps": num_inference_steps,
    }

    if len(instances) == 1 and num_samples > 1:
        instances = instances * num_samples

    response = get_endpoint(endpoint_name).predict(
        instances=instances, parameters=parameters
    )
    images = [
        common_util.base64_to_image(prediction["output"])
        for prediction in response.predictions
    ]

    return images


def inpaint_generate_images(
    endpoint_name: str,
    style_name=None,
    prompt="",
    negative_prompt="",
    num_samples=1,
    guidance_scale=7.5,
    num_inference_steps=25,
    dict=None,
    image_dimension=512,
) -> list[Image.Image]:
    if not endpoint_name:
        raise gr.Error("Select (or deploy) a model first!")

    prompt, negative_prompt = apply_style(style_name, prompt, negative_prompt)
    default_dimension = 512
    # Set the default_dimension=1024 if the model is `stable-diffusion-xl-1.0-inpainting-0.1`.
    if "stable-diffusion-xl" in endpoint_name:
        default_dimension = 1024

    init_image = (
        dict["background"]
        .convert(mode="RGB")
        .resize((default_dimension, default_dimension))
    )
    # Replace the transparent pixels with white. Required in rgba -> rgb conversion.
    mask_rgba = np.array(dict["layers"][0])
    mask_rgba[mask_rgba[..., -1] == 0] = [255, 255, 255, 0]
    mask = Image.fromarray(mask_rgba).convert(mode="L")
    mask = ImageOps.invert(mask).resize((default_dimension, default_dimension))

    instances = [
        {
            "prompt": prompt,
            "negative_prompt": negative_prompt,
            "image": common_util.image_to_base64(init_image),
            "mask_image": common_util.image_to_base64(mask),
            "height": image_dimension,
            "width": image_dimension,
            "guidance_scale": guidance_scale,
            "num_inference_steps": num_inference_steps,
        },
    ]

    if len(instances) == 1 and num_samples > 1:
        instances = instances * num_samples

    response = get_endpoint(endpoint_name).predict(instances=instances)
    images = [common_util.base64_to_image(image) for image in response.predictions]

    return images


def instruct_pix2pix_generate_images(
    endpoint_name: str,
    style_name=None,
    prompt="",
    negative_prompt="",
    num_samples=1,
    guidance_scale=7.5,
    num_inference_steps=25,
    init_image=None,
    image_dimension=512,
) -> list[Image.Image]:
    if not endpoint_name:
        raise gr.Error("Select (or deploy) a model first!")

    prompt, negative_prompt = apply_style(style_name, prompt, negative_prompt)
    instances = [
        {
            "prompt": prompt,
            "negative_prompt": negative_prompt,
            "image": common_util.image_to_base64(init_image),
            "height": image_dimension,
            "width": image_dimension,
            "guidance_scale": guidance_scale,
            "num_inference_steps": num_inference_steps,
        },
    ]

    if len(instances) == 1 and num_samples > 1:
        instances = instances * num_samples

    response = get_endpoint(endpoint_name).predict(instances=instances)
    images = [common_util.base64_to_image(image) for image in response.predictions]

    return images


def upscaler_generate_images(
    endpoint_name: str,
    style_name=None,
    prompt="",
    negative_prompt="",
    num_samples=1,
    guidance_scale=7.5,
    num_inference_steps=25,
    init_image=None,
    image_dimension=512,
) -> list[Image.Image]:

    if not endpoint_name:
        raise gr.Error("Select (or deploy) a model first!")

    prompt, negative_prompt = apply_style(style_name, prompt, negative_prompt)
    default_dimension = 256
    init_image = init_image.convert("RGB").resize(
        (default_dimension, default_dimension)
    )
    instances = [
        {
            "prompt": prompt,
            "negative_prompt": negative_prompt,
            "image": common_util.image_to_base64(init_image),
            "height": image_dimension,
            "width": image_dimension,
            "guidance_scale": guidance_scale,
            "num_inference_steps": num_inference_steps,
        },
    ]

    if len(instances) == 1 and num_samples > 1:
        instances = instances * num_samples

    response = get_endpoint(endpoint_name).predict(instances=instances)
    images = [common_util.base64_to_image(image) for image in response.predictions]

    return images


def controlnet_generate_images(
    endpoint_name: str,
    style_name=None,
    prompt="",
    negative_prompt="",
    num_samples=1,
    guidance_scale=7.5,
    num_inference_steps=25,
    init_image=None,
    image_dimension=512,
    canny_low_threshold=100,
    canny_high_threshold=200,
) -> list[Image.Image]:
    if not endpoint_name:
        raise gr.Error("Select (or deploy) a model first!")

    prompt, negative_prompt = apply_style(style_name, prompt, negative_prompt)
    init_image = init_image.convert("RGB").resize((image_dimension, image_dimension))
    canny_image = canny(init_image, canny_low_threshold, canny_high_threshold)

    instances = [
        {
            "prompt": prompt,
            "negative_prompt": negative_prompt,
            "image": common_util.image_to_base64(canny_image),
            "height": image_dimension,
            "width": image_dimension,
            "guidance_scale": guidance_scale,
            "num_inference_steps": num_inference_steps,
        },
    ]

    if len(instances) == 1 and num_samples > 1:
        instances = instances * num_samples

    response = get_endpoint(endpoint_name).predict(instances=instances)
    images = [common_util.base64_to_image(image) for image in response.predictions]

    images.insert(0, canny_image)

    return images


###############################
#   Image Generation          #
###############################
def create_generation_workshop():
    tip_text = r"""
        1. Select a Vertex prediction endpoint with a model deployed for your chosen task. Mismatched models can lead to unreliable outcomes.
        2. New model deployment takes ~20 minutes. You can check the progress at [Vertex Online Prediction](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).
        3. After the model deployment is complete, restart the playground in Colab to see the updated endpoint list.
        """

    def select_interface(interface_name: str) -> gr.Blocks:
        if interface_name == "Text2Image":
            return {
                image_input: gr.update(visible=False, value=None, label="Upload"),
                inpainting_input: gr.update(visible=False),
                image_output: gr.update(value=None),
                generate_button: gr.update(visible=True),
                inpaint_generate_button: gr.update(visible=False),
                instruct_pix2pix_generate_button: gr.update(visible=False),
                controlnet_generate_button: gr.update(visible=False),
                upscaler_generate_button: gr.update(visible=False),
                canny_low_threshold: gr.update(visible=False),
                canny_high_threshold: gr.update(visible=False),
            }

        elif interface_name == "Inpainting":
            return {
                image_input: gr.update(visible=False),
                inpainting_input: gr.update(visible=True, value=None, label="Upload"),
                image_output: gr.update(value=None),
                generate_button: gr.update(visible=False),
                inpaint_generate_button: gr.update(visible=True),
                instruct_pix2pix_generate_button: gr.update(visible=False),
                controlnet_generate_button: gr.update(visible=False),
                upscaler_generate_button: gr.update(visible=False),
                canny_low_threshold: gr.update(visible=False),
                canny_high_threshold: gr.update(visible=False),
            }

        elif interface_name == "Instruct pix2pix":
            return {
                image_input: gr.update(visible=True, value=None, label="Upload"),
                inpainting_input: gr.update(visible=False),
                image_output: gr.update(value=None),
                generate_button: gr.update(visible=False),
                inpaint_generate_button: gr.update(visible=False),
                instruct_pix2pix_generate_button: gr.update(visible=True),
                controlnet_generate_button: gr.update(visible=False),
                upscaler_generate_button: gr.update(visible=False),
                canny_low_threshold: gr.update(visible=False),
                canny_high_threshold: gr.update(visible=False),
            }

        elif interface_name == "ControlNet Canny":
            return {
                image_input: gr.update(
                    visible=True,
                    value=None,
                    label="Upload a reference image",
                ),
                inpainting_input: gr.update(visible=False),
                image_output: gr.update(value=None),
                generate_button: gr.update(visible=False),
                inpaint_generate_button: gr.update(visible=False),
                instruct_pix2pix_generate_button: gr.update(visible=False),
                controlnet_generate_button: gr.update(visible=True),
                upscaler_generate_button: gr.update(visible=False),
                canny_low_threshold: gr.update(visible=True),
                canny_high_threshold: gr.update(visible=True),
            }

        elif interface_name == "SD 4x Upscaler":
            return {
                image_input: gr.update(visible=True, value=None, label="Upload"),
                inpainting_input: gr.update(visible=False),
                image_output: gr.update(value=None),
                generate_button: gr.update(visible=False),
                inpaint_generate_button: gr.update(visible=False),
                instruct_pix2pix_generate_button: gr.update(visible=False),
                controlnet_generate_button: gr.update(visible=False),
                upscaler_generate_button: gr.update(visible=True),
                canny_low_threshold: gr.update(visible=False),
                canny_high_threshold: gr.update(visible=False),
            }

    def update_default_parameters(model_name: str):
        """Updates the default inference parameters based on the selected model."""
        return {
            guidance_scale: gr.update(value=get_default_guidance_scale(model_name)),
            num_inference_steps: gr.update(
                value=get_default_num_inference_steps(model_name)
            ),
            image_dimension: gr.update(value=get_default_dimension(model_name)),
        }

    with gr.Blocks() as demo:
        interfaces_box = gr.Radio(
            show_label=False,
            choices=[
                "Text2Image",
                "Inpainting",
                "Instruct pix2pix",
                "ControlNet Canny",
                "SD 4x Upscaler",
            ],
            value="Text2Image",
        )
        with gr.Accordion("How To Use", open=False):
            gr.Markdown(tip_text)

        with gr.Row(equal_height=True):
            with gr.Column(scale=3):
                prompt = gr.Textbox(label="Prompt", lines=1)
                negative_prompt = gr.Textbox(label="Negative Prompt", lines=1)
            with gr.Column(scale=1):
                endpoint_name = gr.Dropdown(
                    label="Select a model previously deployed on Vertex",
                    choices=list_endpoints(),
                    value=None,
                )
                with gr.Row():
                    selected_model = gr.Dropdown(
                        scale=7,
                        label="Deploy a new model to Vertex",
                        choices=[
                            "txt2img: stabilityai/stable-diffusion-2-1",
                            "txt2img: stabilityai/stable-diffusion-xl-base-1.0",
                            "txt2img: stabilityai/stable-diffusion-xl-base-1.0 - refiner",
                            "txt2img: latent-consistency/lcm-sdxl",
                            "txt2img: latent-consistency/lcm-lora-sdxl",
                            "txt2img: stabilityai/sdxl-turbo",
                            "txt2img: bytedance/sdxl-lightning",
                            "txt2img: runDiffusion/juggernaut-xl-lightning",
                            "txt2img: stablediffusionapi/juggernaut-xl-v9",
                            "txt2img: lykon/dreamshaper-xl-v2-turbo",
                            "txt2img: stablediffusionapi/anything-xl",
                            "txt2img: stablediffusionapi/turbovision_xl",
                            "inpainting: kandinsky-community/kandinsky-2-2-decoder-inpaint",
                            "inpainting: diffusers/stable-diffusion-xl-1.0-inpainting-0.1",
                            "instruct-pix2pix: timbrooks/instruct-pix2pix",
                            "controlnet: lllyasviel/sd-controlnet-canny",
                            "upscaler: stabilityai/stable-diffusion-x4-upscaler",
                        ],
                        value=None,
                    )
                    deploy_model_button = gr.Button(
                        "Deploy", scale=1, variant="primary", min_width=10
                    )

        with gr.Row(equal_height=True):
            with gr.Column(scale=2):
                generate_button = gr.Button("Generate", variant="primary")
                inpaint_generate_button = gr.Button(
                    "Generate", variant="primary", visible=False
                )
                instruct_pix2pix_generate_button = gr.Button(
                    "Generate", variant="primary", visible=False
                )
                controlnet_generate_button = gr.Button(
                    "Generate", variant="primary", visible=False
                )
                upscaler_generate_button = gr.Button(
                    "Generate", variant="primary", visible=False
                )

                num_samples = gr.Slider(
                    label="Number of samples", value=1, step=1, minimum=1, maximum=4
                )
                image_dimension = gr.Slider(
                    label="Image dimension",
                    value=768,
                    step=256,
                    minimum=512,
                    maximum=1024,
                )
                num_inference_steps = gr.Slider(
                    label="Sampling steps", value=25, step=1, minimum=1, maximum=100
                )
                guidance_scale = gr.Slider(
                    label="Guidance scale", value=7.5, step=0.5, minimum=0, maximum=20.0
                )
                with gr.Accordion("Styles", open=False):
                    style_selection = gr.Radio(
                        show_label=True,
                        container=True,
                        interactive=True,
                        choices=STYLE_NAMES,
                        value=DEFAULT_STYLE_NAME,
                        label="Image Style",
                    )
                    canny_low_threshold = gr.Slider(
                        label="Canny low threshold",
                        value=100,
                        step=5,
                        minimum=1,
                        maximum=255,
                        visible=False,
                    )
                    canny_high_threshold = gr.Slider(
                        label="Canny high threshold",
                        value=200,
                        step=5,
                        minimum=1,
                        maximum=255,
                        visible=False,
                    )

            with gr.Column(scale=5):
                with gr.Row(equal_height=True):
                    image_input = gr.Image(
                        type="pil",
                        label="Upload",
                        sources="upload",
                        height=500,
                        interactive=True,
                        visible=False,
                    )
                    inpainting_input = gr.ImageMask(
                        type="pil",
                        label="Upload",
                        sources="upload",
                        eraser=None,
                        height=500,
                        interactive=True,
                        visible=False,
                    )
                    image_output = gr.Gallery(
                        label="Generated Images",
                        rows=1,
                        height=500,
                        preview=True,
                    )

    interfaces_box.change(
        select_interface,
        interfaces_box,
        [
            image_input,
            inpainting_input,
            image_output,
            generate_button,
            inpaint_generate_button,
            instruct_pix2pix_generate_button,
            controlnet_generate_button,
            upscaler_generate_button,
            canny_low_threshold,
            canny_high_threshold,
        ],
    )

    endpoint_name.change(
        update_default_parameters,
        endpoint_name,
        [
            guidance_scale,
            num_inference_steps,
            image_dimension,
        ],
    )

    deploy_model_button.click(
        deploy_model,
        inputs=[selected_model],
        outputs=[],
    )

    generate_button.click(
        generate_images,
        inputs=[
            endpoint_name,
            style_selection,
            prompt,
            negative_prompt,
            num_samples,
            guidance_scale,
            num_inference_steps,
            image_dimension,
        ],
        outputs=image_output,
    )

    inpaint_generate_button.click(
        inpaint_generate_images,
        inputs=[
            endpoint_name,
            style_selection,
            prompt,
            negative_prompt,
            num_samples,
            guidance_scale,
            num_inference_steps,
            inpainting_input,
            image_dimension,
        ],
        outputs=image_output,
    )

    instruct_pix2pix_generate_button.click(
        instruct_pix2pix_generate_images,
        inputs=[
            endpoint_name,
            style_selection,
            prompt,
            negative_prompt,
            num_samples,
            guidance_scale,
            num_inference_steps,
            image_input,
            image_dimension,
        ],
        outputs=image_output,
    )

    controlnet_generate_button.click(
        controlnet_generate_images,
        inputs=[
            endpoint_name,
            style_selection,
            prompt,
            negative_prompt,
            num_samples,
            guidance_scale,
            num_inference_steps,
            image_input,
            image_dimension,
            canny_low_threshold,
            canny_high_threshold,
        ],
        outputs=image_output,
    )

    upscaler_generate_button.click(
        upscaler_generate_images,
        inputs=[
            endpoint_name,
            style_selection,
            prompt,
            negative_prompt,
            num_samples,
            guidance_scale,
            num_inference_steps,
            image_input,
            image_dimension,
        ],
        outputs=image_output,
    )
    return demo


###############################
#   Dreambooth finetuning     #
###############################
def pad_image(image):
    w, h = image.size
    if w == h:
        return image
    elif w > h:
        new_image = Image.new(image.mode, (w, w), (0, 0, 0))
        new_image.paste(image, (0, (w - h) // 2))
        return new_image
    else:
        new_image = Image.new(image.mode, (h, h), (0, 0, 0))
        new_image.paste(image, ((h - w) // 2, 0))
        return new_image


def get_dreambooth_base_model_and_resolution(base_model_select: str) -> tuple[str, int]:
    if base_model_select == "sd-v2-1-768":
        return "stabilityai/stable-diffusion-2-1", 768
    elif base_model_select == "sd-v2-1-512":
        return "stabilityai/stable-diffusion-2-1", 512
    elif base_model_select == "sdxl-dreambooth-lora-1024":
        return "stabilityai/stable-diffusion-xl-base-1.0", 1024
    else:
        raise gr.Error(f"Invalid base model: {base_model_select}")


# The pre-built training docker image. They contain training scripts and models.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-train:20240318_0936_RC00"


def create_dreambooth_workshop() -> gr.Blocks:
    def prepare_instance_images(
        base_model_select: str,
        concept_prompt: str,
        file_collection: list[gr.File],
    ):
        """
        Prepares the instance images for training from the provided dir of images.
        """
        if not file_collection:
            raise gr.Error(gr.Markdown("Provide a few valid instance images!"))

        if not concept_prompt:
            raise gr.Error("Provide a unique concept prompt!")

        resolution = 768
        if base_model_select == "sd-v2-1-512" or base_model_select == "sd-v1-5-512":
            resolution = 512
        elif base_model_select == "sdxl-dreambooth-lora-1024":
            resolution = 1024

        local_tmp_dir = "/tmp/instance_images"
        if os.path.exists(local_tmp_dir):
            shutil.rmtree(local_tmp_dir)
        os.makedirs(local_tmp_dir, exist_ok=True)

        image_counter = 0
        for file_temp in file_collection:
            image_counter += 1
            file = Image.open(file_temp.name)
            image = pad_image(file)
            image = image.resize((resolution, resolution))
            image = image.convert("RGB")
            image.save(
                os.path.join(local_tmp_dir, f"{concept_prompt}_{image_counter}.jpg"),
                format="JPEG",
                quality=100,
            )

        # Upload the processed images to the GCS dir.
        instant_image_dir = os.path.join(BUCKET_URI, f"instance_images-{now}")
        upload_local_dir_to_gcs(local_tmp_dir, instant_image_dir)

    def deploy_finetuned_model(
        base_model_select: str, output_dir: str
    ) -> aiplatform.Endpoint:
        if base_model_select == "sdxl-dreambooth-lora-1024":
            return deploy_model_vertex(
                model_id="stabilityai/stable-diffusion-xl-base-1.0",
                lora_id=output_dir,
                task_name="text-to-image-sdxl",
                use_dedicated_endpoint=True,
            )

        return deploy_model_vertex(
            model_id=output_dir,
            task_name="text-to-image",
            use_dedicated_endpoint=use_dedicated_endpoint,
        )

    def dreambooth_start_training(*inputs):
        [
            output_dir,
            base_model_select,
            concept_prompt,
            learning_rate,
            train_steps,
            deploy_after_tuning,
            train_text_encoder,
            checkpointing_steps,
            train_batch_size,
            gradient_accumulation_steps,
            lr_scheduler,
            lr_warmup_steps,
            seed,
        ] = inputs[:13]

        if not concept_prompt:
            raise gr.Error("Enter a concept prompt first!")

        if not inputs or len(inputs) < 13:
            raise gr.Error("Provide a local folder with (5-10) instance image!")

        if not base_model_select:
            raise gr.Error("Select a base model!")

        gr.Info(f"Dreambooth finetune with task {base_model_select} started!")

        file_collection = list(chain.from_iterable(inputs[13:]))
        base_model_id, resolution = get_dreambooth_base_model_and_resolution(
            base_model_select
        )

        image_dir_gcs = os.path.join(BUCKET_URI, f"instance_images-{now}")
        image_dir_gcsfuse = image_dir_gcs.replace("gs://", "/gcs/")
        tuned_model_dir_gcs = output_dir
        tuned_model_dir_gcsfuse = tuned_model_dir_gcs.replace("gs://", "/gcs/")

        print("The tuned model will be saved at:", output_dir)
        output_dir_bucket = "/".join(output_dir.split("/")[:3])  # noqa: F841
        # Provision permissions to the two SERVICE_ACCOUNTs with the provided GCS bucket
        ! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $output_dir_bucket
        ! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT_CC}:roles/storage.admin $output_dir_bucket

        prepare_instance_images(
            base_model_select=base_model_select,
            concept_prompt=concept_prompt,
            file_collection=list(file_collection),
        )

        # Worker pool spec.
        machine_type = "g2-standard-8"
        num_nodes = 1
        gpu_type = "NVIDIA_L4"
        num_gpus = 1

        common_util.check_quota(
            project_id=PROJECT_ID,
            region=REGION,
            accelerator_type=gpu_type,
            accelerator_count=num_gpus,
            is_for_training=True,
        )

        # Add labels for the finetuning job.
        labels = {
            "mg-source": "notebook",
            "mg-notebook-name": "model_garden_pytorch_stable_diffusion_gradio.ipynb".split(
                "."
            )[
                0
            ],
        }

        labels["mg-tune"] = "publishers-stability-ai-models-sd"
        versioned_model_id = base_model_select
        labels["versioned-mg-tune"] = f"{labels['mg-tune']}-{versioned_model_id}"

        # Setup training job.
        job_name = common_util.create_job_name(f"{base_model_id}-dreambooth")
        job = aiplatform.CustomContainerTrainingJob(
            display_name=job_name,
            container_uri=TRAIN_DOCKER_URI,
            labels=labels,
        )

        gr.Info(f"Custom Training job {job_name} is being submitted. Let's wait.")

        task = "text-to-image-dreambooth"
        if base_model_select == "sdxl-dreambooth-lora-1024":
            task = "text-to-image-dreambooth-lora-sdxl"

        args = [
            f"--task={task}",
            f"--pretrained_model_name_or_path={base_model_id}",
            f"--instance_data_dir={image_dir_gcsfuse}",
            f"--class_data_dir={image_dir_gcsfuse}",
            f"--output_dir={tuned_model_dir_gcsfuse}",
            f"--instance_prompt={concept_prompt}",
            f"--resolution={int(resolution)}",
            f"--train_batch_size={int(train_batch_size)}",
            f"--gradient_accumulation_steps={int(gradient_accumulation_steps)}",
            f"--learning_rate={learning_rate}",
            f"--lr_scheduler={lr_scheduler}",
            f"--lr_warmup_steps={int(lr_warmup_steps)}",
            f"--max_train_steps={int(train_steps)}",
            f"--checkpointing_steps={int(checkpointing_steps)}",
            f"--seed={int(seed)}",
            "--gradient_checkpointing",
            "--mixed_precision=fp16",
            "--use_8bit_adam",
        ]

        if base_model_select == "sdxl-dreambooth-lora-1024":
            args.append(
                "--pretrained_vae_model_name_or_path=madebyollin/sdxl-vae-fp16-fix"
            )
        if train_text_encoder:
            args.append("--train_text_encoder")

        job.run(
            args=args,
            replica_count=num_nodes,
            machine_type=machine_type,
            accelerator_type=gpu_type,
            accelerator_count=num_gpus,
            sync=True,
        )

        if deploy_after_tuning:
            deploy_finetuned_model(base_model_select, output_dir)

    def update_default_lr(base_model_select: str):
        """Updates the default learning rate based on the selected base model."""
        lr = 2e-6
        if base_model_select == "sdxl-dreambooth-lora-1024":
            lr = 1e-4

        return {
            learning_rate: gr.update(value=lr),
        }

    def update_default_checkpoing_steps(train_steps: int):
        """Updates the default checkpointing_steps based on the training steps."""
        return {checkpointing_steps: gr.update(value=int(train_steps // 2))}

    dreambooth_tip_text = r"""
        1. Upload 5-10 images of the object/face on from different angles/perspectives.
        2. Dreambooth training overfits easily. You need to tune the learning_rate and train_steps to get the best results.
           Refer to the [Dreambooth Tips](https://huggingface.co/blog/dreambooth) for more details.
        3. After the training job started, you can check the job status at
           [Vertex Custom Training](https://console.cloud.google.com/vertex-ai/training/custom-jobs?project={PROJECT_ID}).
        4. If you choose to deploy to Vertex after tuning, you can check the deployed endpoint at [Vertex Online Prediction](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints?project={PROJECT_ID}).
        """

    with gr.Blocks() as demo:
        with gr.Accordion("How To Use", open=False):
            gr.Markdown(dreambooth_tip_text)
        with gr.Row(equal_height=True):
            with gr.Column():
                base_model_select = gr.Dropdown(
                    label="Select a base model",
                    choices=[
                        "sd-v2-1-768",
                        "sd-v2-1-512",
                        "sd-v1-5-512",
                        "sdxl-dreambooth-lora-1024",
                    ],
                    value="sd-v2-1-768",
                    interactive=True,
                )
                concept_prompt = gr.Textbox(
                    label="Concept prompt - use a unique, made up word to avoid collisions",
                    lines=1,
                )
                learning_rate = gr.Number(
                    label="Learning rate", value=2e-6, interactive=True
                )
                train_steps = gr.Number(
                    label="Number of training steps", value=800, interactive=True
                )
                deploy_after_tuning = gr.Checkbox(
                    label="Deploy to Vertex after tuning", value=True, interactive=True
                )

                with gr.Accordion("Advanced Settings", open=False):
                    train_text_encoder = gr.Checkbox(
                        label="Train text encoder", value=False, interactive=True
                    )
                    checkpointing_steps = gr.Number(
                        label="Checkpointing steps", value=400, interactive=True
                    )
                    train_batch_size = gr.Number(
                        label="Training batch size", value=1, interactive=True
                    )
                    gradient_accumulation_steps = gr.Number(
                        label="Gradient accumulation steps", value=1, interactive=True
                    )
                    lr_scheduler = gr.Dropdown(
                        label="Learning rate scheduler",
                        choices=[
                            "constant",
                            "linear",
                            "cosine",
                            "cosine_with_restarts",
                            "polynomial",
                            "constant_with_warmup",
                            "piecewise_costant",
                        ],
                        value="constant",
                    )
                    lr_warmup_steps = gr.Number(
                        label="Warmup steps", value=0, interactive=True
                    )
                    seed = gr.Number(label="Seed", value=42, interactive=True)

            with gr.Column():
                train_button = gr.Button("Start Training", variant="primary")

                file_collection = []
                file_collection.append(
                    gr.File(
                        label="Upload 5-10 reference images for your concept",
                        file_types=["image"],
                        file_count="multiple",
                        interactive=True,
                        visible=True,
                    )
                )

                output_dir_gcs = os.path.join(BUCKET_URI, f"tuned_model-{now}")
                output_dir = gr.Textbox(
                    label="The GCS directory to save the tuned model. Defaults to the staging bucket set/created at initiation.",
                    value=output_dir_gcs,
                    lines=1,
                    interactive=True,
                    visible=True,
                )

    base_model_select.change(
        update_default_lr,
        base_model_select,
        [
            learning_rate,
        ],
    )

    train_steps.change(
        update_default_checkpoing_steps,
        train_steps,
        [
            checkpointing_steps,
        ],
    )

    train_button.click(
        dreambooth_start_training,
        inputs=[
            output_dir,
            base_model_select,
            concept_prompt,
            learning_rate,
            train_steps,
            deploy_after_tuning,
            train_text_encoder,
            checkpointing_steps,
            train_batch_size,
            gradient_accumulation_steps,
            lr_scheduler,
            lr_warmup_steps,
            seed,
        ]
        + file_collection,
        outputs=[],
    )

    return demo


css = """
    .gradio-container {
      width: 90% !important
    }
"""
with gr.Blocks(
    css=css, theme=gr.themes.Default(primary_hue="orange", secondary_hue="blue")
) as demo:
    gr.Markdown("# Model Garden Playground for Image Generation")

    with gr.Tabs():
        with gr.TabItem("Image Generation"):
            create_generation_workshop()
        with gr.TabItem("Dreambooth Finetune"):
            create_dreambooth_workshop()

show_debug_logs = True  # @param {type: "boolean"}
demo.queue()
demo.launch(share=True, inline=False, debug=show_debug_logs, show_error=True)

## Clean up resources

In [ ]:
# @markdown Delete temporary GCS buckets.

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME